# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# display(trips.take(5))
display(zone_names.take(30))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    
    #using only the required columns from the trips dataset
    req_col = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
    df = df.select(*req_col)
    
    # filtering out rows based on the below condition
    df = df.filter(~((df.trip_distance == 0) | ((df.passenger_count == 0) &(df.total_amount <= 0))))
    return df
    

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 2,858,133


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


In [0]:
# defining a function for Median Absolute function for easy processing later and avoid code repetation
def mad(df):
    median_amt = df.agg(pf.percentile_approx("total_amount", 0.5)).collect()[0][0]
    median_distance = df.agg(pf.percentile_approx("trip_distance", 0.5)).collect()[0][0]
#     func1_amt = (pf.percentile_approx(pf.abs(df.total_amount - median_amt))
#     func2_dist = (pf.percentile_approx(pf.abs(df.trip_distance - median_distance))
    mad_amt = 1.483 * df.agg(pf.percentile_approx(pf.abs(df.total_amount - median_amt), 0.5)).collect()[0][0]
    mad_dist = 1.483 * df.agg(pf.percentile_approx(pf.abs(df.trip_distance - median_distance), 0.5)).collect()[0][0]
    
    return median_amt,median_distance,mad_amt,mad_dist

    

In [0]:
# defining zscore function
def zscore(df,median_amt,median_distance,mad_amt,mad_dist):
    zscore_amt = (df.total_amount - median_amt)/mad_amt
    zscore_dist = (df.trip_distance - median_distance)/mad_dist
    
    return zscore_amt,zscore_dist

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    
    median_amt,median_distance,mad_amt,mad_dist = mad(df) # Calling the mad() function 
    zscore_amt,zscore_dist = zscore(df,median_amt,median_distance,mad_amt,mad_dist) # Calling the zscore() function
    
    # adding the columns names, zscore_amt and zscore_dist
    df = df.withColumn('zscore_amt', zscore_amt) 
    df = df.withColumn('zscore_dist', zscore_dist)
    
    # filtering out outliers using the below condition
    df = df.filter((df.zscore_amt) <= 3.5) 
    df = df.filter((df.zscore_dist) <= 3.5) 
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,502,072


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
display(trips_12.take(10))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,zscore_amt,zscore_dist
90,68,0.8,1.0,8.8,-1.0288749806096642,-0.7807786492529365
113,90,0.9,1.0,8.8,-1.0288749806096642,-0.7097987720481241
88,232,2.8,1.0,13.8,-0.23743268783299942,0.6388188948433117
79,249,1.4,1.0,12.3,-0.47486537566599885,-0.35489938602406207
142,238,2.0,0.0,12.3,-0.47486537566599885,0.07097987720481248
114,90,1.6,1.0,12.8,-0.3957211463883324,-0.21293963161443713
90,144,1.8,1.0,13.3,-0.3165769171106659,-0.07097987720481232
114,48,2.0,1.0,14.75,-0.08705865220543324,0.07097987720481248
48,152,5.7,1.0,22.3,1.1080192098873307,2.697235333782872
234,148,1.8,1.0,14.75,-0.08705865220543324,-0.07097987720481232


In [0]:
display(zone_names.take(10))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    
    # Applying left join on LocationID of zone_df=zone_names and PULocationID from df=output of 1.2
    df = df.join(zones_df.select("LocationID","Zone"),df.PULocationID ==  zones_df.LocationID,"left")
    
    # Renaming column as start
    df = df.withColumnRenamed('Zone', 'start')
    df = df.drop("LocationID")  # Avoid duplication
    df = df.join(zones_df.select("LocationID","Zone"),df.DOLocationID ==  zones_df.LocationID,"left")
    
    # Renaming column as End
    df = df.withColumnRenamed('Zone', 'end')
    df = df.drop("LocationID")
    return df
    

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 2,502,072


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,zscore_amt,zscore_dist,start,end
90,68,0.8,1.0,8.8,-1.0288749806096642,-0.7807786492529365,Flatiron,East Chelsea
113,90,0.9,1.0,8.8,-1.0288749806096642,-0.7097987720481241,Greenwich Village North,Flatiron
88,232,2.8,1.0,13.8,-0.23743268783299942,0.6388188948433117,Financial District South,Two Bridges/Seward Park
79,249,1.4,1.0,12.3,-0.47486537566599885,-0.35489938602406207,East Village,West Village
142,238,2.0,0.0,12.3,-0.47486537566599885,0.07097987720481248,Lincoln Square East,Upper West Side North
114,90,1.6,1.0,12.8,-0.3957211463883324,-0.21293963161443713,Greenwich Village South,Flatiron
90,144,1.8,1.0,13.3,-0.3165769171106659,-0.07097987720481232,Flatiron,Little Italy/NoLiTa
114,48,2.0,1.0,14.75,-0.08705865220543324,0.07097987720481248,Greenwich Village South,Clinton East
48,152,5.7,1.0,22.3,1.1080192098873307,2.697235333782872,Clinton East,Manhattanville
234,148,1.8,1.0,14.75,-0.08705865220543324,-0.07097987720481232,Union Sq,Lower East Side


In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    unit_profitabilty = df.total_amount / df.trip_distance
    df = df.withColumn('unit_profitabilty', unit_profitabilty)
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
display(trips_22.take(10))

Row count: 2,502,072


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,zscore_amt,zscore_dist,start,end,unit_profitabilty
90,68,0.8,1.0,8.8,-1.0288749806096642,-0.7807786492529365,Flatiron,East Chelsea,11.0
113,90,0.9,1.0,8.8,-1.0288749806096642,-0.7097987720481241,Greenwich Village North,Flatiron,9.777777777777779
88,232,2.8,1.0,13.8,-0.23743268783299942,0.6388188948433117,Financial District South,Two Bridges/Seward Park,4.928571428571429
79,249,1.4,1.0,12.3,-0.47486537566599885,-0.35489938602406207,East Village,West Village,8.785714285714286
142,238,2.0,0.0,12.3,-0.47486537566599885,0.07097987720481248,Lincoln Square East,Upper West Side North,6.15
114,90,1.6,1.0,12.8,-0.3957211463883324,-0.21293963161443713,Greenwich Village South,Flatiron,8.0
90,144,1.8,1.0,13.3,-0.3165769171106659,-0.07097987720481232,Flatiron,Little Italy/NoLiTa,7.388888888888889
114,48,2.0,1.0,14.75,-0.08705865220543324,0.07097987720481248,Greenwich Village South,Clinton East,7.375
48,152,5.7,1.0,22.3,1.1080192098873307,2.697235333782872,Clinton East,Manhattanville,3.912280701754386
234,148,1.8,1.0,14.75,-0.08705865220543324,-0.07097987720481232,Union Sq,Lower East Side,8.194444444444445


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    
    # creating  anew dataframe categorised by PULocationID', 'DOLocationID and calculating the average unit profitability,the total number of recorded trips and the total number of passenger across all trips. 
    graph_df = df.groupBy('PULocationID', 'DOLocationID').agg(pf.avg('unit_profitabilty'), pf.count('trip_distance'), pf.sum('passenger_count'))
    
    #average unit profitability between two zones
    graph_df = graph_df.withColumnRenamed("avg(unit_profitabilty)", 'average_unit_profit') 
    
    #the total number of recorded trips between any two zones
    graph_df = graph_df.withColumnRenamed("count(trip_distance)" , 'trips_count')
    
    #the total number of passenger across all trips between any two zones
    graph_df = graph_df.withColumnRenamed("sum(passenger_count)", 'total_passengers')

    return graph_df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 13,622


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
90,231,7.4193267864052315,1124,1577.0
87,33,6.817160729833857,105,145.0
90,142,6.8603295530133215,908,1277.0
114,100,7.480600705110231,465,705.0
234,144,9.119099004361532,1488,2187.0
148,262,4.840744491138079,280,415.0
246,249,8.807349712997269,1151,1714.0
170,179,5.138416672570028,79,102.0
48,232,5.345169208403833,360,526.0
142,144,5.766557628964919,351,492.0


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    df = df.join(zones_df.select("LocationID","Zone"),df.PULocationID ==  zones_df.LocationID,"left")
    df = df.withColumnRenamed('Zone', 'start')
    df = df.drop("LocationID")
    df = df.join(zones_df.select("LocationID","Zone"),df.DOLocationID ==  zones_df.LocationID,"left")
    df = df.withColumnRenamed('Zone', 'end')
    df = df.drop("LocationID")
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.groupBy('start').agg(pf.sum('trips_count'))
    df_zones = df_zones.withColumnRenamed("sum(trips_count)", 'Total_trips_by_Zone')
    df_zones = df_zones.sort(pf.desc('Total_trips_by_Zone')).head(10)
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.groupBy('start').agg(pf.avg('average_unit_profit'))
    df_zones = df_zones.withColumnRenamed("avg(average_unit_profit)", 'avg_profit_by_zone')
    df_zones = df_zones.sort(pf.desc('avg_profit_by_zone')).head(10)

    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    df_zones = df_zones.groupBy('start').agg(pf.sum('total_passengers'))
    df_zones = df_zones.withColumnRenamed("sum(total_passengers)", 'total_passengers_by_zone')
    df_zones = df_zones.sort(pf.desc('total_passengers_by_zone')).head(10)
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

start,Total_trips_by_Zone
Upper East Side South,122777
Upper East Side North,104451
Penn Station/Madison Sq West,97524
Midtown Center,94357
Murray Hill,89133
Midtown East,85890
Lincoln Square East,81530
Upper West Side South,75364
Clinton East,75223
Lenox Hill West,74939


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

start,avg_profit_by_zone
Bay Terrace/Fort Totten,311.9330683658483
Baisley Park,99.14984180611125
Prospect Park,90.58902850606826
Saint Michaels Cemetery/Woodside,66.15323958582769
Van Cortlandt Village,48.69059979610944
Grymes Hill/Clifton,38.65
Crotona Park,36.84437137879828
Flatbush/Ditmas Park,32.90238757671382
Eastchester,31.2081577880231
Flushing Meadows-Corona Park,28.941580387822043


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

start,total_passengers_by_zone
Upper East Side South,171228.0
Upper East Side North,144963.0
Penn Station/Madison Sq West,138288.0
Midtown Center,135975.0
Murray Hill,124771.0
Midtown East,120803.0
Lincoln Square East,115508.0
Clinton East,108161.0
Upper West Side South,106775.0
East Village,106558.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'delta'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)
print(trips.count())


    Trips dataset loaded!
    ---
      Size: S
      Format: delta
      Tables loaded: /FileStore/tables/taxi/delta/taxi-S-delta/
      Number of trips (dataset rows): 2,898,033
    
2898033


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[30]: [[Row(start='Upper East Side South', Total_trips_by_Zone=122777),
  Row(start='Upper East Side North', Total_trips_by_Zone=104451),
  Row(start='Penn Station/Madison Sq West', Total_trips_by_Zone=97524),
  Row(start='Midtown Center', Total_trips_by_Zone=94357),
  Row(start='Murray Hill', Total_trips_by_Zone=89133),
  Row(start='Midtown East', Total_trips_by_Zone=85890),
  Row(start='Lincoln Square East', Total_trips_by_Zone=81530),
  Row(start='Upper West Side South', Total_trips_by_Zone=75364),
  Row(start='Clinton East', Total_trips_by_Zone=75223),
  Row(start='Lenox Hill West', Total_trips_by_Zone=74939)],
 [Row(start='Bay Terrace/Fort Totten', avg_profit_by_zone=311.9330683658483),
  Row(start='Baisley Park', avg_profit_by_zone=99.14984180611125),
  Row(start='Prospect Park', avg_profit_by_zone=90.58902850606826),
  Row(start='Saint Michaels Cemetery/Woodside', avg_profit_by_zone=66.15323958582769),
  Row(start='Van Cortlandt Village', avg_profit_by_zone=48.69059979610944)

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2.89 |15.57 |41.95 |90.44 |132.39|
| execution time   (w/o 1.2)  |6.63 seconds  |1.46 min |13.91 min|19.52 min |18.67 min |
| execution time              |13.72 seconds | 3.27 min | 27.87 min |38.51 min | 42.59 min |
| sec / 1M records (w/o 1.2)  |2.294 | 5.626  | 19.895 |12.950 |8.461  |
| sec / 1M records            |4.747 | 12.601 |39.861 |25.54 | 19.302 |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2.89 |15.57 |41.95 |90.44 |132.39|
| execution time   (w/o 1.2)  |15.41 seconds |8.02 seconds  |11.42 seconds |17.64 seconds |25.80 seconds |
| execution time              |12.25 seconds |19.92 seconds |21.75 seconds |30.14 seconds |44.39 seconds |
| sec / 1M records (w/o 1.2)  |5.332 |0.515 |0.272 |0.195 |0.194 |
| sec / 1M records            |4.238 |1.279 |0.518 |0.333 |0.335 |